# GeoAlchemy2 ORM Test

Source: [https://geoalchemy-2.readthedocs.io/en/latest/orm_tutorial.html](https://geoalchemy-2.readthedocs.io/en/latest/orm_tutorial.html)

In [ ]:
import psycopg2, yaml

with open("config.yaml", "r") as f:
    config = yaml.safe_load(f)

conn_string = "host=%(SERVER)s user=%(USERNAME)s dbname=%(DATABASE)s password=%(PASSWORD)s sslmode=%(SSLMODE)s" % config
conn = psycopg2.connect(conn_string) 
print("Connection established")

## Connect to the DB

In [ ]:
from sqlalchemy import create_engine

engine = create_engine(
    "postgresql+psycopg2://%(USERNAME)s:%(PASSWORD)s@%(SERVER)s/%(DATABASE)s?sslmode=%(SSLMODE)s" % config,
    client_encoding="utf8", 
    echo=True
)

## Declare a Mapping

We will try to map the CSD digital boundary files 

In [ ]:
from sqlalchemy.orm import declarative_base
from sqlalchemy import Column, Integer, String, Float
from geoalchemy2 import Geometry

Base = declarative_base()

class CSDTable(Base):
    __tablename__ = "CSD_DBF_2021"
    DGUID = Column(String, primary_key=True)
    CSDUID = Column(Integer)
    CSDNAME = Column(String)
    CSDTYPE = Column(String)
    LANDAREA = Column(Float)
    geometry = Column(Geometry('POLYGON'))

In [ ]:
import geopandas as gpd
import os

# import the 2021 Digital Boundary File
csd_dbf = gpd.read_file(os.path.join(os.path.abspath(""), "data", "lcsd000a21a_e.zip"))

# project to lat/lon

csd_dbf = csd_dbf.to_crs("epsg:4326")

csd_dbf["CSDUID"] = csd_dbf["CSDUID"].astype(int) 
csd_dbf["PRUID"]  = csd_dbf["PRUID"].astype(int)
csd_dbf["LANDAREA"]  = csd_dbf["LANDAREA"].astype(float)

csd_dbf

In [ ]:
csd_dbf.info()